In [1]:

#Get set up
import requests
import json
import pandas as pd
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

#from tqdm import tqdm_notebook as tqdm
import time

In [26]:
#open the codebook and put it in a data dictionary
codebook = pd.read_csv('RawCodeBook.csv',  encoding='ISO-8859-1', delimiter = ',')
tmp_cb = codebook[['Category', 'Variable Name']]
tmp_cb = tmp_cb.set_index('Variable Name')
codebook_dict = tmp_cb.to_dict()

In [127]:
codebook_dict['Category']['felony']


'Prohibitions for high-risk gun possession'

In [138]:
#now, open up the main file
gv_df = pd.read_csv('RawLawsAndDeaths.csv',  encoding='ISO-8859-1', delimiter = ',')


In [140]:
#loop through the rows and then columns, creating 1 record per state, year, and law type column. 
#There is a look up to get the law type at the category level
gv_output = pd.DataFrame(columns=['state','year','crude','category', 'value'])
l = list(gv_df.columns.values)
for j in range (0, len(gv_df)):
    for i in l:
        #print(i)
        if i == 'state':
            v_state = gv_df[i][j]
        elif i == 'year':
            v_year = gv_df[i][j]
        elif i == 'Crude Rate':
            v_crude = gv_df[i][j] 
        elif i == 'lawtotal':
            v_lawtotal=gv_df[i][j]
        else:
            v_cat = codebook_dict['Category'][i]
#            print(v_cat + str(gv_df[i][j]))
            gv_output=gv_output.append({'state':v_state, 'year':v_year,'crude':v_crude, 'category':v_cat,'value':gv_df[i][j]}, ignore_index=True)



In [142]:
gv_output

,state,year,crude,category,value
0,Wyoming,2017,19.5,Prohibitions for high-risk gun possession,1
1,Wyoming,2017,19.5,Prohibitions for high-risk gun possession,0
2,Wyoming,2017,19.5,Prohibitions for high-risk gun possession,0
3,Wyoming,2017,19.5,Prohibitions for high-risk gun possession,0
4,Wyoming,2017,19.5,Prohibitions for high-risk gun possession,0
...,...,...,...,...,...
120595,Hawaii,2004,3.2,Domestic violence,1
120596,Hawaii,2004,3.2,Domestic violence,1
120597,Hawaii,2004,3.2,Domestic violence,1
120598,Hawaii,2004,3.2,Domestic violence,0


In [143]:
#wrote this to excel to excel for validation
gv_output.to_csv('gv_out_big.csv', index=False)

In [145]:
len(gv_output)

120600

In [147]:
#Dedupe to a new dataframe
gv_dedupe = gv_output.drop_duplicates(subset=None, keep='first', inplace=False)

In [148]:
len(gv_dedupe)

17782

In [151]:
#manually checked to make sure deduping did not cause problems
gv_dedupe.to_csv('gv_dedupe.csv', index=False)

In [153]:
#Remove 0's.  I checked, there is no state/year combo that does not have at least 1 category marked as 1
gv_pos = gv_dedupe[gv_dedupe['value']==1]

In [154]:
gv_pos

,state,year,crude,category,value
0,Wyoming,2017,19.5,Prohibitions for high-risk gun possession,1
80,Wyoming,2017,19.5,Possession regulations,1
94,Wyoming,2017,19.5,Stand your ground,1
109,Wyoming,2017,19.5,Immunity,1
134,Wisconsin,2017,10.8,Prohibitions for high-risk gun possession,1
...,...,...,...,...,...
120560,Hawaii,2004,3.2,Stand your ground,1
120567,Hawaii,2004,3.2,Child access prevention,1
120575,Hawaii,2004,3.2,Immunity,1
120576,Hawaii,2004,3.2,Preemption,1


In [157]:
#The data is in the right format for tableau but the crude rate repeats for every different law category. This will cause our 
#crude rate to be inflated.  We need to set the records to 0 if there is more than 1 category per year and state.
gv_pos_sorted = gv_pos.sort_values(['state', 'year'])


In [158]:
gv_pos_sorted.head(3)

,state,year,crude,category,value
21044,Alabama,2000,17.2,Prohibitions for high-risk gun possession,1
21050,Alabama,2000,17.2,Dealer regulations,1
21125,Alabama,2000,17.2,Concealed carry permitting,1


In [185]:
prev_state =''
prev_year =''
for i in range(0, len(gv_pos_sorted)):
    if (prev_state == gv_pos_sorted['state'].iloc[i] and prev_year ==  gv_pos_sorted['year'].iloc[i]):
        gv_pos_sorted.iloc[i,2]=-0
    prev_state = gv_pos_sorted['state'].iloc[i]
    prev_year =  gv_pos_sorted['year'].iloc[i]

In [186]:
gv_pos_sorted.head(50)

,state,year,crude,category,value
21044,Alabama,2000,17.2,Prohibitions for high-risk gun possession,1
21050,Alabama,2000,0.0,Dealer regulations,1
21125,Alabama,2000,0.0,Concealed carry permitting,1
21132,Alabama,2000,0.0,Stand your ground,1
21171,Alabama,2000,0.0,Domestic violence,1
23590,Alabama,2001,16.5,Prohibitions for high-risk gun possession,1
23596,Alabama,2001,0.0,Dealer regulations,1
23671,Alabama,2001,0.0,Concealed carry permitting,1
23678,Alabama,2001,0.0,Stand your ground,1
23717,Alabama,2001,0.0,Domestic violence,1


In [187]:
#wrote this to excel to excel for validation
gv_pos_sorted.to_csv('gv.csv', index=False)